# Automated ML


In [1]:
!pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 25 kB/s s eta 0:00:01K     |██████████████████▏             | 81.3 MB 81.1 MB/s eta 0:00:01█████████████████████████████▏| 138.9 MB 67.7 MB/s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3


In [115]:
import json
import sys
import os
import numpy as np
import pandas as pd
import shutil
import joblib
import requests

from sklearn.model_selection import train_test_split

from TrainCovid19Infections import clean_data

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.run import Run
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.train.automl import constants

## Initialize Workspace

In [3]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
# choose a name for experiment
experiment_name = 'Covid19VaccinationExperiment'
experiment=Experiment(ws, experiment_name)
experiment

Workspace name: quick-starts-ws-141116
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-141116


Name,Workspace,Report Page,Docs Page
Covid19VaccinationExperiment,quick-starts-ws-141116,Link to Azure Machine Learning studio,Link to Documentation


## Create Cluster

Get cluster if it exists else create one

In [4]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

Creating a new compute target...
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-22T12:01:12.223000+00:00', 'errors': None, 'creationTime': '2021-03-22T12:01:09.763985+00:00', 'modifiedTime': '2021-03-22T12:01:25.544576+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

I Chose a COVID-19 World Wide infections Dataset that holds a track of Covid19's propagation rate, new cases, patients with chronic deseases and death rate per country. 

Since the covid-19 pandemic is among the hottest subjects in the world, and as a member of the society being interested in such statistics calculation can help further scientists or even regular people to better understand the global effect of this deadly virus all over the world.

I used Github to download the Dataset.

In [127]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset2"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True)
    
    
    #ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,...,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00,76411.00
mean,667890.62,5133.14,5070.62,17147.36,114.34,113.27,7920.17,66.60,65.57,161.58,...,27.70,7.88,5.02,17412.73,236.16,7.25,23.49,2.55,69.50,0.67
std,4720571.35,32192.62,31656.62,108601.10,674.30,652.60,15745.49,168.45,141.42,316.81,...,12.42,6.48,4.37,19622.91,133.76,4.30,33.33,2.52,17.53,0.25
min,0.00,-74347.00,-6223.00,0.00,-1918.00,-232.14,0.00,-2153.44,-276.82,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,679.00,1.00,4.43,10.00,0.00,0.00,124.42,0.06,0.74,1.63,...,19.60,3.01,1.78,2896.91,140.45,4.61,0.00,0.70,66.47,0.56
50%,7871.00,55.00,63.14,130.00,1.00,0.86,1043.30,5.88,7.57,19.71,...,29.00,5.44,3.21,10727.15,233.07,6.93,0.00,2.00,74.16,0.74
75%,87467.50,663.00,669.00,1720.00,11.00,11.57,7236.47,56.07,62.12,140.37,...,38.00,13.26,8.35,25063.85,318.95,9.75,47.78,3.60,78.49,0.83
max,123207571.00,880902.00,739564.43,2715295.00,17903.00,14431.43,149058.43,8652.66,2648.77,2327.77,...,48.20,27.05,18.49,116935.60,724.42,30.53,99.00,13.80,86.75,0.96


In [117]:
# preview the first 10 rows of the dataset
df.head(10)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.00,1.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
1,AFG,Asia,Afghanistan,2020-02-25,1.00,0.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
2,AFG,Asia,Afghanistan,2020-02-26,1.00,0.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
3,AFG,Asia,Afghanistan,2020-02-27,1.00,0.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
4,AFG,Asia,Afghanistan,2020-02-28,1.00,0.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
5,AFG,Asia,Afghanistan,2020-02-29,1.00,0.00,0.14,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
6,AFG,Asia,Afghanistan,2020-03-01,1.00,0.00,0.14,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
7,AFG,Asia,Afghanistan,2020-03-02,1.00,0.00,0.00,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
8,AFG,Asia,Afghanistan,2020-03-03,2.00,1.00,0.14,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51
9,AFG,Asia,Afghanistan,2020-03-04,4.00,2.00,0.43,0.00,0.00,0.00,...,1803.99,0,597.03,9.59,0,0,37.75,0.50,64.83,0.51


In [128]:
# Use the clean_data function to clean your data.
x, y = clean_data(df)
print(x)
data = pd.concat([x,y],axis=1)
data.head(10)

         date  total_cases  new_cases_smoothed  total_deaths  new_deaths  \
0      737479         1.00                0.00          0.00        0.00   
1      737480         1.00                0.00          0.00        0.00   
2      737481         1.00                0.00          0.00        0.00   
3      737482         1.00                0.00          0.00        0.00   
4      737483         1.00                0.00          0.00        0.00   
...       ...          ...                 ...           ...         ...   
76406  737866     36552.00               30.14       1508.00        1.00   
76407  737867     36611.00               33.43       1509.00        1.00   
76408  737868     36652.00               32.71       1510.00        1.00   
76409  737869     36662.00               27.29       1510.00        0.00   
76410  737870     36665.00               25.86       1512.00        2.00   

       new_deaths_smoothed  total_cases_per_million  new_cases_per_million  \
0        

,date,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,new_cases
0,737479,1.00,0.00,0.00,0.00,0.00,0.03,0.03,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
1,737480,1.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
2,737481,1.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
3,737482,1.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
4,737483,1.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
5,737484,1.00,0.14,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
6,737485,1.00,0.14,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
7,737486,1.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
8,737487,2.00,0.14,0.00,0.00,0.00,0.05,0.03,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
9,737488,4.00,0.43,0.00,0.00,0.00,0.10,0.05,0.01,0.00,...,0,0,0,0,1,0,0,0,0,1


In [129]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.5,random_state = 42,shuffle=True)

In [9]:
# Create necessary folders
if "automl_training" not in os.listdir():
    os.mkdir("./automl_training")
if "data" not in os.listdir():
    os.mkdir("./data")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")
# store training_dataset into it using datastore
script_folder = './automl_training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', script_folder)
project_folder = './pipeline-project'

## AutoML Configuration
TODO: Explain why you chose the automl settings and cofiguration you used below.
The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions 

In [10]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('training/training_data.csv')
#Create an experiment for the AutoML testing script
exp = Experiment(workspace=ws, name="Covid19AutoMlExperiment")

# Get the dataset from the data folder
datastore.upload_files(files = ['training/training_data.csv'],
                       target_path ='./data/',
                       overwrite = True,
                       show_progress = True)
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('./data/training_data.csv'))])
#training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1.0,
    "max_concurrent_iterations": 3,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = cpu_cluster,
                             training_data = training_dataset,
                             label_column_name = 'new_cases',
                             featurization= 'auto',
                             path=project_folder,
                              model_explainability=True,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [11]:
# Experiment Submission
tag = {"Covid19Infections": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

Running on remote.
No run_configuration provided, running on Covid19Cluster with default configuration
Running on remote compute: Covid19Cluster
Parent Run ID: AutoML_60eae57c-bfa4-4f4f-a29e-7ecea2bfc5b3

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?
Supervised Learning in general is based on labled 

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_60eae57c-bfa4-4f4f-a29e-7ecea2bfc5b3',
 'target': 'Covid19Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-22T12:02:39.323129Z',
 'endTimeUtc': '2021-03-22T13:35:48.464707Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Covid19Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Covid19VaccinationExperiment","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-141116","workspace_name":"quick-starts-ws-141116","region":"southcentralus","compute_target":"Covid19Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":

In [13]:
# List best models of HyperDrive Run and AutoML Run to compare the accuracy of the models and choose the best among them to deploy
for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t',prop_name,':',prop)
    print("\n")

## Best Model

explaining_model_run_id = remote_run.id
print(explaining_model_run_id)
explaining_model_run = Run(experiment=experiment, run_id=explaining_model_run_id)
explaining_model_run.wait_for_completion()

In [14]:
# Retrieve and save best model.
best_automl_run, best_automl_model = remote_run.get_output()

In [15]:
# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()

# Print all metrics of the best run model
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print(best_automl_model._final_estimator)

f1_score_macro 0.9987926351127925
accuracy 0.9990653576672812
balanced_accuracy 0.9987678249612376
recall_score_weighted 0.9990653576672812
recall_score_macro 0.9987678249612376
AUC_micro 0.9999681399599165
norm_macro_recall 0.9975356499224752
weighted_accuracy 0.9992524220936128
log_loss 0.021352105573640742
average_precision_score_macro 0.9999242086410094
average_precision_score_micro 0.9999686013044201
f1_score_micro 0.9990653576672812
AUC_macro 0.999956660728485
recall_score_micro 0.9990653576672812
precision_score_macro 0.9988177452009885
precision_score_micro 0.9990653576672812
AUC_weighted 0.999956660728485
precision_score_weighted 0.9990654799224539
average_precision_score_weighted 0.999953330604682
f1_score_weighted 0.9990653241184582
matthews_correlation 0.9975855148956834
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_60eae57c-bfa4-4f4f-a29e-7ecea2bfc5b3_21/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_60eae57c-bfa4-4f4f-a29e-7ecea2b

In [16]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Covid19VaccinationExperiment,AutoML_60eae57c-bfa4-4f4f-a29e-7ecea2bfc5b3_21,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
joblib.dump(best_automl_model, filename="./outputs/automl-model.pkl")

['./outputs/automl-model.pkl']

In [18]:
arr = os.listdir('./outputs/')
print(arr)

['automl-model.pkl', 'model.joblib']


In [19]:
from azureml.core.model import Model
model = Model.list(ws)
print (model)

[]


In [20]:
# register the best model
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
best_model_registered = remote_run.register_model(
                                           
                                            model_name = "Covid19-automl-model",
                                            tags={'Area': "Pandemic", 'Type': "Classification",'Method of execution':'Auto ML'},
                                            )
#print(best_model.name, best_model.id, sep='\t')
print(best_model_registered)

Model(workspace=Workspace.create(name='quick-starts-ws-141116', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-141116'), name=Covid19-automl-model, id=Covid19-automl-model:1, version=1, tags={'Area': 'Pandemic', 'Type': 'Classification', 'Method of execution': 'Auto ML'}, properties={})


## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
with open('score.py') as f:
    print(f.read())

import json
import numpy as np
import os
import joblib
import pandas as pd

def init():
    #This function initialises the model. The model file is retrieved used within the script.
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automl_covid19_model.pkl')
    print("Found model:", os.path.isfile(model_path))
    model = joblib.load(model_path)
    print ("model initialized" + time.strftime("%H:%M:%S"))

def run(data):
    try:
        data = np.array(json.loads(data))
        #data = np.array(json.loads(data))
        #data = json.loads(data)['data'] # raw = pd.read_json(data) 
        #data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error


In [22]:
from azureml.automl.core.shared import constants

model = Model(ws, 'Covid19-automl-model')

environment = best_automl_run.get_environment()
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'environment.yml')


inference_config = InferenceConfig(entry_script='score.py', environment=environment)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='Covid19 new cases prediction',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)

service = Model.deploy(workspace=ws,
                       name='aci-covid19-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-22 13:39:04+00:00 Creating Container Registry if not exists.
2021-03-22 13:39:05+00:00 Use the existing image.
2021-03-22 13:39:05+00:00 Generating deployment configuration.
2021-03-22 13:39:06+00:00 Submitting deployment to compute.
2021-03-22 13:39:09+00:00 Checking the status of deployment aci-covid19-service..
2021-03-22 13:41:46+00:00 Checking the status of inference endpoint aci-covid19-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [23]:
print("Key " + service.get_keys()[0])
print("Swagger URI : "+service.swagger_uri)
print("Scoring URI : "+service.scoring_uri)

Key ISBAEXz6ORSOCVzlrZtfO5BKxJzEEwtl
Swagger URI : http://bd43c1a9-1445-4d43-922a-5d90dc372caf.southcentralus.azurecontainer.io/swagger.json
Scoring URI : http://bd43c1a9-1445-4d43-922a-5d90dc372caf.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [109]:
#Importing the dataset for testing 
#training_data,validation_data = train_test_split(data,test_size = 0.3,random_state = 42,shuffle=True
train_data,test_data = train_test_split(data,test_size = 0.3,random_state = 42,shuffle=True)

TypeError: Expected sequence or array-like, got <class 'NoneType'>

In [148]:
sample_data = validation_data.sample(2)
y_test = sample_data["new_cases"]
sample_data.drop(['new_cases'], inplace=True, axis=1)
x_test = sample_data.sample(2)
x_test.head(10)

,date,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,iso_code_ZAF,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America
33284,737792,597033.00,893.43,12829.00,5.00,8.86,14843.26,20.88,22.21,318.95,...,0,0,0,0,0,1,0,0,0,0
20167,737843,235882.00,759.86,3048.00,10.00,12.71,21744.48,38.72,70.05,280.98,...,0,0,0,0,0,0,0,1,0,0


In [149]:
Covid19DataTesting= json.dumps({'data': x_test.to_dict(orient='records')})
print(Covid19DataTesting)

{"data": [{"date": 737792, "total_cases": 597033.0, "new_cases_smoothed": 893.429, "total_deaths": 12829.0, "new_deaths": 5.0, "new_deaths_smoothed": 8.857, "total_cases_per_million": 14843.258, "new_cases_per_million": 20.884, "new_cases_smoothed_per_million": 22.212, "total_deaths_per_million": 318.951, "new_deaths_per_million": 0.124, "new_deaths_smoothed_per_million": 0.22, "reproduction_rate": 0.81, "icu_patients": 0, "icu_patients_per_million": 0, "hosp_patients": 0, "new_tests": 0, "total_tests": "4618587.0", "total_tests_per_thousand": "114.826", "positive_rate": "0.025", "tests_per_case": "39.9", "stringency_index": 53.7, "population": 40222503.0, "population_density": 88.125, "median_age": 20.0, "aged_65_older": 3.186, "aged_70_older": 1.957, "gdp_per_capita": 15663.986, "extreme_poverty": "2.5", "cardiovasc_death_rate": 218.612, "diabetes_prevalence": 8.83, "female_smokers": 0, "male_smokers": 0, "handwashing_facilities": 94.576, "hospital_beds_per_thousand": 1.4, "life_expe

In [150]:
for col in x_test.columns:
    print (col)

date
total_cases
new_cases_smoothed
total_deaths
new_deaths
new_deaths_smoothed
total_cases_per_million
new_cases_per_million
new_cases_smoothed_per_million
total_deaths_per_million
new_deaths_per_million
new_deaths_smoothed_per_million
reproduction_rate
icu_patients
icu_patients_per_million
hosp_patients
new_tests
total_tests
total_tests_per_thousand
positive_rate
tests_per_case
stringency_index
population
population_density
median_age
aged_65_older
aged_70_older
gdp_per_capita
extreme_poverty
cardiovasc_death_rate
diabetes_prevalence
female_smokers
male_smokers
handwashing_facilities
hospital_beds_per_thousand
life_expectancy
human_development_index
tests_units
iso_code_AFG
iso_code_AGO
iso_code_AIA
iso_code_ALB
iso_code_AND
iso_code_ARE
iso_code_ARG
iso_code_ARM
iso_code_ATG
iso_code_AUS
iso_code_AUT
iso_code_AZE
iso_code_BDI
iso_code_BEL
iso_code_BEN
iso_code_BFA
iso_code_BGD
iso_code_BGR
iso_code_BHR
iso_code_BHS
iso_code_BIH
iso_code_BLR
iso_code_BLZ
iso_code_BMU
iso_code_BOL
iso

In [151]:
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {service.get_keys()[0]}'
# Make the request and display the response
response = requests.post(service.scoring_uri, Covid19DataTesting, headers=headers)
print('Prediction :', response.text)

# Print original labels
print('True Values :', y_test.values)

Prediction : Invalid input data frame: data frame has size 260 on dimension #1, while expected value is 261
True Values : [1 1]


In [ ]:
print(service.get_logs())

In [ ]:
#service.delete()
#cpu_cluster.delete()